In [1]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

import funciones_auxiliares as aux

# warnings.filterwarnings('ignore')
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
df = pd.read_csv('income-db.csv')

In [3]:
df_depurada = df.replace('?', np.nan)
df_depurada = df_depurada.dropna()

In [37]:
REEMPLAZO_OCCUPATION = {'white-collar': ['Prof-specialty', 'Exec-managerial', 'Adm-clerical', 'Sales',
                                         'Tech-support'],
                        'blue-collar': ['Craft-repair', 'Machine-op-inspct', 'Transport-moving',
                                        'Handlers-cleaners', 'Farming-fishing', 'Protective-serv',
                                        'Priv-house-serv'],
                        'others': ['Other-service', 'Armed-Forces']}

REEMPLAZO_WORKCLASS = {'federal-gov': ['Federal-gov'],
                       'state-level-gov': ['State-gov', 'Local-gov'],
                       'self-employed': ['Self-emp-inc', 'Self-emp-not-inc'],
                       'unemployed': ['Never-worked', 'Without-pay']}

REEMPLAZO_EDUCATION = {'preschool': ['Preschool'],
                       'elementary-school': ['1st-4th', '5th-6th'],
                       'high-school': ['7th-8th', '9th', '10th','11th', '12th', 'HS-grad'],
                       'college': ['Assoc-voc', 'Assoc-acdm', 'Some-college'],
                       'university': ['Bachelors', 'Masters', 'Prof-school', 'Doctorate']}

REEMPLAZO_MARITAL = {'married': ['Married-civ-spouse', 'Married-spouse-absent','Married-AF-spouse'],
                     'divorced': ['Divorced'],
                     'separated': ['Separated'],
                     'widowed': ['Widowed']}

REEMPLAZO_COUNTRY = {'america': ["United-States", "Mexico", "Puerto-Rico", "Canada", "El-Salvador", 
                                 "Cuba", "Jamaica", "Dominican-Republic", "Guatemala", "Columbia", 
                                 "Haiti", "Nicaragua", "Peru", "Ecuador", "Trinadad&Tobago", 
                                 "Outlying-US(Guam-USVI-etc)"],
                     'asia': ["Philippines", "India", "China", "Japan", "Vietnam", "Taiwan", 
                              "Iran", "Hong", "Thailand", "Cambodia", "Laos"],
                     'europe': ["Germany", "England", "Italy", "Poland", "Portugal", "Greece", 
                                "France", "Ireland", "Yugoslavia", "Scotland", "Honduras", 
                                "Hungary", "Holand-Netherlands"],
                     'oceania': [],
                     'africa': ["South"]}

REEMPLAZO_INCOME = {0: ['<=50K'], 1: ['>50K']}
REEMPLAZO_SEX = {0: ['Male'], 1: ['Female']}

RECODIFICACION_ENUNCIADO_1 = [REEMPLAZO_OCCUPATION, REEMPLAZO_WORKCLASS, REEMPLAZO_EDUCATION,
                              REEMPLAZO_MARITAL, REEMPLAZO_COUNTRY, REEMPLAZO_INCOME, REEMPLAZO_SEX]

def recodificar_variable(df, diccionario_cambio):
    tmp = df.copy()
    for valor_nuevo, valores_antiguos in diccionario_cambio.items():
        tmp = tmp.replace(valores_antiguos, valor_nuevo)

    return tmp

def recodificar_enunciado_uno(df):
    recodificada = df.copy()
    for dict_recod in RECODIFICACION_ENUNCIADO_1:
        recodificada = recodificar_variable(recodificada, dict_recod)
    
    return recodificada


In [38]:
recodificar_enunciado_uno(df_depurada)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,high-school,7,Never-married,blue-collar,Own-child,Black,0,0,0,40,america,0
1,38,Private,89814,high-school,9,married,blue-collar,Husband,White,0,0,0,50,america,0
2,28,state-level-gov,336951,college,12,married,blue-collar,Husband,White,0,0,0,40,america,1
3,44,Private,160323,college,10,married,blue-collar,Husband,Black,0,7688,0,40,america,1
5,34,Private,198693,high-school,6,Never-married,others,Not-in-family,White,0,0,0,30,america,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,college,12,married,white-collar,Wife,White,1,0,0,38,america,0
48838,40,Private,154374,high-school,9,married,blue-collar,Husband,White,0,0,0,40,america,1
48839,58,Private,151910,high-school,9,widowed,white-collar,Unmarried,White,1,0,0,40,america,0
48840,22,Private,201490,high-school,9,Never-married,white-collar,Own-child,White,0,0,0,20,america,0
